In [1]:
from pathlib import Path
import typer
from spacy.tokens import DocBin
import spacy
from pandas import read_csv
import random
from spacy.training.iob_utils import offsets_to_biluo_tags, biluo_tags_to_spans
import csv
import re

In [2]:
with open("assets/bottins.csv", "r", newline="", encoding="utf-8") as csv_file:
    csv_reader = csv.reader(csv_file)
    with open('bottins.txt', 'w', encoding='utf-8', errors='replace') as txt_file:
            for row in csv_reader:
                print(row)
                ligne = ','.join(row)
                txt_file.write(ligne + '\n')

['<PER>Dufan et Clémendot</PER>, <ACT>pharmaciens</ACT>, <LOC>r. de la\u2029Chaussée-d&apos;Antin</LOC>, <CARDINAL>34</CARDINAL>. <TITRE>(Elig.)</TITRE> 449', ' "Bottin1_1820"']
['<PER>Dufant (Victor)</PER>, <ACT>libraire</ACT>, <LOC>r. du Gros-Che-\u2029net</LOC>, <CARDINAL>2</CARDINAL>. 392', ' "Bottin1_1820"']
['<PER>Dufay</PER>, <ACT>essayeur du commerce</ACT>, <LOC>place Dau-\u2029phine</LOC>, <CARDINAL>5</CARDINAL>.         355', ' "Bottin1_1820"']
['<PER>Dulay</PER>, <ACT>chandronnier</ACT>, <LOC>r. du Pont- aux\u2029Choux</LOC>, <CARDINAL>15</CARDINAL>. 314', ' "Bottin1_1820"']
['<PER>Dufay (V.e)</PER>, <ACT>grenetière</ACT>, <LOC>r. du Faub.-S.\u2029Denis</LOC>, <CARDINAL>20</CARDINAL>. 372', ' "Bottin1_1820"']
['<PER>Dufay</PER>, <ACT>papetier</ACT>, <LOC>r. S.-Martin</LOC>, <CARDINAL>20</CARDINAL>. <CARDINAL>437</CARDINAL>', ' "Bottin1_1820"']
['<PER>Dufeu</PER>, <ACT>charcutier</ACT>, <LOC>r. Montmartre</LOC>, <CARDINAL>89</CARDINAL>. 318', ' "Bottin1_1820"']
['<PER>Dufey f

In [3]:
with open("bottins.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()
    print(lines)

['<PER>Dufan et Clémendot</PER>, <ACT>pharmaciens</ACT>, <LOC>r. de la\u2029Chaussée-d&apos;Antin</LOC>, <CARDINAL>34</CARDINAL>. <TITRE>(Elig.)</TITRE> 449, "Bottin1_1820"\n', '<PER>Dufant (Victor)</PER>, <ACT>libraire</ACT>, <LOC>r. du Gros-Che-\u2029net</LOC>, <CARDINAL>2</CARDINAL>. 392, "Bottin1_1820"\n', '<PER>Dufay</PER>, <ACT>essayeur du commerce</ACT>, <LOC>place Dau-\u2029phine</LOC>, <CARDINAL>5</CARDINAL>.         355, "Bottin1_1820"\n', '<PER>Dulay</PER>, <ACT>chandronnier</ACT>, <LOC>r. du Pont- aux\u2029Choux</LOC>, <CARDINAL>15</CARDINAL>. 314, "Bottin1_1820"\n', '<PER>Dufay (V.e)</PER>, <ACT>grenetière</ACT>, <LOC>r. du Faub.-S.\u2029Denis</LOC>, <CARDINAL>20</CARDINAL>. 372, "Bottin1_1820"\n', '<PER>Dufay</PER>, <ACT>papetier</ACT>, <LOC>r. S.-Martin</LOC>, <CARDINAL>20</CARDINAL>. <CARDINAL>437</CARDINAL>, "Bottin1_1820"\n', '<PER>Dufeu</PER>, <ACT>charcutier</ACT>, <LOC>r. Montmartre</LOC>, <CARDINAL>89</CARDINAL>. 318, "Bottin1_1820"\n', '<PER>Dufey fils</PER>, <AC

In [4]:
data = []

In [5]:
for ligne in lines:
    mots = re.split(r",", ligne)

    per = None
    loc = None
    cardinal = None
    act = None
    titre = None

    for mot in mots:
        if "<PER>" in mot:
            per = mot
        elif "<LOC>" in mot:
            loc = mot
        elif "<CARDINAL>" in mot:
            cardinal = mot
        elif "<ACT>" in mot:
            act = mot
        elif "<TITRE>" in mot:
            titre = mot

    random_elements_line = ""
    elements = [per, act, loc, cardinal, titre]
    random.shuffle(elements)

    for i in range(0, 5):
        random_elements_line += f"{elements[i].strip()} " if elements[i] is not None else ""

    matches = re.findall(
        r"<PER>(.*?)<\/PER>|<ACT>(.*?)<\/ACT>|<LOC>(.*?)<\/LOC>|<CARDINAL>(\d+)<\/CARDINAL>|<TITRE>(.*?)<\/TITRE>",
        random_elements_line)

    entities = []
    start_pos = 0

    sentences = ""
    for match in matches:

        for i, m in enumerate(match):
            if m:
                tag = ["PER", "ACT", "LOC", "CARDINAL", "TITRE"][i]
                end_pos = start_pos + len(str(m))
                entities.append((start_pos, end_pos, tag))
                start_pos = end_pos + 1
                
                sentences += f'{m}'

    data_tuple = (sentences, entities)
    data.append(data_tuple)

In [6]:
data

[('r. de la\u2029Chaussée-d&apos;AntinpharmaciensDufan et Clémendot34(Elig.)',
  [(0, 30, 'LOC'),
   (31, 42, 'ACT'),
   (43, 61, 'PER'),
   (62, 64, 'CARDINAL'),
   (65, 72, 'TITRE')]),
 ('2Dufant (Victor)librairer. du Gros-Che-\u2029net',
  [(0, 1, 'CARDINAL'), (2, 17, 'PER'), (18, 26, 'ACT'), (27, 46, 'LOC')]),
 ('essayeur du commerceplace Dau-\u2029phineDufay5',
  [(0, 20, 'ACT'), (21, 37, 'LOC'), (38, 43, 'PER'), (44, 45, 'CARDINAL')]),
 ('r. du Pont- aux\u2029Chouxchandronnier15Dulay',
  [(0, 21, 'LOC'), (22, 34, 'ACT'), (35, 37, 'CARDINAL'), (38, 43, 'PER')]),
 ('20Dufay (V.e)grenetièrer. du Faub.-S.\u2029Denis',
  [(0, 2, 'CARDINAL'), (3, 14, 'PER'), (15, 25, 'ACT'), (26, 46, 'LOC')]),
 ('20437papetierr. S.-MartinDufay',
  [(0, 2, 'CARDINAL'),
   (3, 6, 'CARDINAL'),
   (7, 15, 'ACT'),
   (16, 28, 'LOC'),
   (29, 34, 'PER')]),
 ('89r. MontmartreDufeucharcutier',
  [(0, 2, 'CARDINAL'), (3, 16, 'LOC'), (17, 22, 'PER'), (23, 33, 'ACT')]),
 ('bijoutierDufey filspassage de la Réunion